In [ ]:
# install umap and friends
!pip install -q umap-learn[plot]
!pip install -q bokeh

from io import BytesIO
import base64
from PIL import Image

from bokeh import plotting, palettes
from bokeh.models import HoverTool, ColumnDataSource, CategoricalColorMapper
import numpy as np
import pandas as pd

plotting.output_notebook()

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader
from torch import nn

from google.colab import drive
drive.mount('/content/drive')

x = torch.load("drive/MyDrive/MNIST Feature Attribution/" + "x_subset.tensor")
y = torch.load("drive/MyDrive/MNIST Feature Attribution/" + "y_subset.tensor")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from io import BytesIO
import base64
from PIL import Image

from bokeh import plotting, palettes
from bokeh.models import HoverTool, ColumnDataSource, CategoricalColorMapper
import numpy as np
import pandas as pd

plotting.output_notebook()

In [ ]:
import umap

reducerFish = umap.UMAP(n_neighbors = 20,min_dist=0.5, verbose = True)
embeddingFish = reducerFish.fit_transform(torch.reshape(x, (50, 2457600)))

UMAP(min_dist=0.5, n_neighbors=20, verbose=True)


/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)


Wed Mar  9 07:20:04 2022 Construct fuzzy simplicial set
Wed Mar  9 07:20:08 2022 Finding Nearest Neighbors
Wed Mar  9 07:20:12 2022 Finished Nearest Neighbor Search
Wed Mar  9 07:20:15 2022 Construct embedding


Epochs completed:   0%|            0/500 [00:00]

Wed Mar  9 07:20:19 2022 Finished embedding


In [ ]:
def embeddableImage(data):
    # data should be [4, 3, 200, 1024] instance
    encoded_subimages = []
    for sub_image in data:
      #print(sub_image.shape)
      #img_data = (255 * (1 - sub_image)).astype(np.uint8) 
      img_data = (255 * sub_image).astype(np.uint8).transpose(1, 2, 0)
      image = Image.fromarray(img_data, mode='RGB')
      buffer = BytesIO()
      image.save(buffer, format='png')
      encoded_subimages.append('data:image/png;base64,' + base64.b64encode(buffer.getvalue()).decode())
    return encoded_subimages

In [ ]:
def umapPlot(embedding, x, y, yTrue=None, title=''):
    """ Plot the embedding of X and y with popovers using Bokeh """
    
    df = pd.DataFrame(embedding, columns=('x', 'y'))
    # for each image should be able to apply the embeddable image function
    # list of lists [rows x columns], x instances with 4 columns
    sub_images = np.array(list(map(embeddableImage, x)))
    for i in range(4):
      df['image'+str(i+1)] = sub_images[:,i]
    df['class'] = [str(d) for d in y]
    df['index'] = list(range(len(y)))
    if yTrue is not None:
        df['trueDigit'] = [str(d) for d in yTrue]

    datasource = ColumnDataSource(df)

    colorMapping = CategoricalColorMapper(factors=np.arange(10).astype(np.str), palette=palettes.Spectral10)

    plotFigure = plotting.figure(
        title=title,
        plot_width=600,
        plot_height=600,
        tools=('pan, wheel_zoom, reset')
    )

    if yTrue is None:
        tooltip = """
            <div>
                <div>
                    <img src='@image1' style='float: left; width:256px; height:50px; margin: 5px 5px 5px 5px'/>
                </div>
                <div>
                    <img src='@image2' style='float: left; width:256px; height:50px; margin: 5px 5px 5px 5px'/>
                </div>
                <div>
                    <img src='@image3' style='float: left; width:256px; height:50px; margin: 5px 5px 5px 5px'/>
                </div>
                <div>
                    <img src='@image4' style='float: left; width:256px; height:50px; margin: 5px 5px 5px 5px'/>
                </div>
                <div>
                    <span style='font-size: 16px; color: #224499'>Class:</span>
                    <span style='font-size: 18px'>@class</span>
                    <span style='font-size: 16px; color: #224499'>Index:</span>
                    <span style='font-size: 18px'>@index</span>
                </div>
            </div>
            """
    else:
        tooltip = """
            <div>
                <div>
                    <img src='@image' style='float: left; margin: 5px 5px 5px 5px'/>
                </div>
                <div>
                    <span style='font-size: 16px; color: #224499'>Digit:</span>
                    <span style='font-size: 18px'>@digit (true: @trueDigit)</span>
                </div>
            </div>
            """
    plotFigure.add_tools(HoverTool(tooltips=tooltip))

    plotFigure.circle(
        'x', 'y',
        source=datasource,
        color=dict(field='class', transform=colorMapping),
        line_alpha=0.6, fill_alpha=0.6, size=8
    )
    plotting.show(plotFigure)
    
    return plotFigure

In [ ]:
fig = umapPlot(embeddingFish, x.numpy(), y.squeeze().numpy(), title='UMAP projection of the Zebrafish dataset')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
